VERSIÓN ÚNICAMENTE CON DEEPFACE

In [ ]:
import cv2

from deepface import DeepFace
 
# Cargar las imágenes de superposición

lipstick_image = cv2.imread('../P5/images/labiosrojos.png', cv2.IMREAD_UNCHANGED)

mustache_image = cv2.imread('../P5/images/mostacho.png', cv2.IMREAD_UNCHANGED)
 
# Verificar si las imágenes se han cargado correctamente

if lipstick_image is None:

    print("Error: No se pudo cargar la imagen de labios.")

if mustache_image is None:

    print("Error: No se pudo cargar la imagen de bigote.")
 
# Convertir las imágenes a BGRA si no tienen canal alfa

if lipstick_image is not None and lipstick_image.shape[2] == 3:

    lipstick_image = cv2.cvtColor(lipstick_image, cv2.COLOR_BGR2BGRA)

if mustache_image is not None and mustache_image.shape[2] == 3:

    mustache_image = cv2.cvtColor(mustache_image, cv2.COLOR_BGR2BGRA)
 
def overlay_image(background, overlay, position):

    x, y = position

    h, w = overlay.shape[:2]
 
    # Superponer donde no hay transparencia en overlay

    alpha_overlay = overlay[:, :, 3] / 255.0  # Canal alfa de overlay

    alpha_background = 1.0 - alpha_overlay
 
    for c in range(0, 3):  # Para cada canal de color

        background[y:y+h, x:x+w, c] = (alpha_overlay * overlay[:, :, c] +

                                       alpha_background * background[y:y+h, x:x+w, c])

    return background
 
# Iniciar la cámara

vid = cv2.VideoCapture(0)
 
while True:

    ret, frame = vid.read()

    if not ret:

        break
 
    try:

        # Análisis del rostro solo con detección de género

        obj = DeepFace.analyze(frame, actions=['gender'], enforce_detection=False)
 
        # Obtener el género de la primera cara detectada

        gender = obj[0]['dominant_gender']  # Verificamos que está en obj[0]
 
        # Detección de la región facial

        faces = DeepFace.extract_faces(frame, detector_backend="retinaface", align=False)

        if len(faces) > 0 and 'facial_area' in faces[0]:

            facial_area = faces[0]['facial_area']

            x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']

            # Aplicar superposición basada en el género

            if gender == 'Man' and lipstick_image is not None:

                overlay_img = cv2.resize(lipstick_image, (w, h // 2))

                frame = overlay_image(frame, overlay_img, (x, y + h // 2))

            elif gender == 'Woman' and mustache_image is not None:

                overlay_img = cv2.resize(mustache_image, (w, h // 4))

                frame = overlay_image(frame, overlay_img, (x, y + h // 2))
 
        # Muestra el frame con la superposición

        cv2.imshow("Filtro de Género", frame)

    except Exception as e:

        print("Error:", e)
 
    # Esc para salir

    if cv2.waitKey(1) & 0xFF == 27:

        break
 
# Cerrar cámara y ventanas

vid.release()

cv2.destroyAllWindows()

 

VERSIÓN CON HAAR CASCADE

In [ ]:
import cv2
import time
from deepface import DeepFace

# Cargar las imágenes de superposición
lipstick_image = cv2.imread('../P5/images/labiosrojos.png', cv2.IMREAD_UNCHANGED)
mustache_image = cv2.imread('../P5/images/mostacho.png', cv2.IMREAD_UNCHANGED)

# Verificar si las imágenes se han cargado correctamente
if lipstick_image is None:
    print("Error: No se pudo cargar la imagen de labios.")
if mustache_image is None:
    print("Error: No se pudo cargar la imagen de bigote.")

# Convertir las imágenes a BGRA si no tienen canal alfa
if lipstick_image is not None and lipstick_image.shape[2] == 3:
    lipstick_image = cv2.cvtColor(lipstick_image, cv2.COLOR_BGR2BGRA)
if mustache_image is not None and mustache_image.shape[2] == 3:
    mustache_image = cv2.cvtColor(mustache_image, cv2.COLOR_BGR2BGRA)

def overlay_image(background, overlay, position):
    """Superponer la imagen overlay sobre la imagen background en la posición indicada."""
    x, y = position
    h, w = overlay.shape[:2]
    alpha_overlay = overlay[:, :, 3] / 255.0
    alpha_background = 1.0 - alpha_overlay

    for c in range(0, 3):  # Para cada canal de color
        background[y:y+h, x:x+w, c] = (alpha_overlay * overlay[:, :, c] +
                                       alpha_background * background[y:y+h, x:x+w, c])
    return background

# Iniciar la cámara
vid = cv2.VideoCapture(0)

# Inicializar variables para el análisis de género
gender = None
last_detection_time = 0
detection_interval = 2  # Intervalo en segundos para detección de género

while True:
    ret, frame = vid.read()
    if not ret:
        break

    current_time = time.time()
    
    # Solo analizar el género cada 'detection_interval' segundos
    if current_time - last_detection_time > detection_interval or gender is None:
        try:
            # Analizamos el género en una versión más pequeña del frame para acelerar el procesamiento
            resized_frame = cv2.resize(frame, (640, 480))
            obj = DeepFace.analyze(resized_frame, actions=['gender'], enforce_detection=False)
            gender = obj[0]['dominant_gender']
            last_detection_time = current_time
        except Exception as e:
            print("Error de análisis:", e)
            continue

    # Usar Haar Cascade de OpenCV para detectar rostros rápidamente
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        # Aplicar el filtro correspondiente según el género detectado
        if gender == 'Man' and lipstick_image is not None:
            # Reducimos un poco el tamaño de los labios y bajamos su posición
            overlay_img = cv2.resize(lipstick_image, (w // 2, h // 5))
            frame = overlay_image(frame, overlay_img, (x + w // 4, y + int(h * 0.7)))
        
        elif gender == 'Woman' and mustache_image is not None:
            # Ajustamos el tamaño y bajamos un poco el bigote
            overlay_img = cv2.resize(mustache_image, (w // 2, h // 6))
            frame = overlay_image(frame, overlay_img, (x + w // 4, y + int(h * 0.65)))

    # Mostrar el frame con la superposición
    cv2.imshow("Filtro de Genero", frame)

    # Salir con 'ESC'
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Cerrar cámara y ventanas
vid.release()
cv2.destroyAllWindows()


VERSIÓN CON MEDIAPIPE

In [ ]:
import cv2
import time
from deepface import DeepFace
import mediapipe as mp

lipstick_image = cv2.imread('../P5/images/labiosrojos.png', cv2.IMREAD_UNCHANGED)
mustache_image = cv2.imread('../P5/images/mostacho.png', cv2.IMREAD_UNCHANGED)

if lipstick_image is None:
    print("Error: No se pudo cargar la imagen de labios.")
if mustache_image is None:
    print("Error: No se pudo cargar la imagen de bigote.")

if lipstick_image is not None and lipstick_image.shape[2] == 3:
    lipstick_image = cv2.cvtColor(lipstick_image, cv2.COLOR_BGR2BGRA)
if mustache_image is not None and mustache_image.shape[2] == 3:
    mustache_image = cv2.cvtColor(mustache_image, cv2.COLOR_BGR2BGRA)

def overlay_image(background, overlay, position):
    x, y = position
    h, w = overlay.shape[:2]
    alpha_overlay = overlay[:, :, 3] / 255.0
    alpha_background = 1.0 - alpha_overlay

    for c in range(0, 3):  
        background[y:y+h, x:x+w, c] = (alpha_overlay * overlay[:, :, c] +
                                       alpha_background * background[y:y+h, x:x+w, c])
    return background

vid = cv2.VideoCapture(0)

gender = None
last_detection_time = 0
detection_interval = 2

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

with mp_face_detection.FaceDetection(min_detection_confidence=0.2) as face_detection:
    while True:
        ret, frame = vid.read()
        if not ret:
            break

        current_time = time.time()
        
        if current_time - last_detection_time > detection_interval or gender is None:
            try:
                resized_frame = cv2.resize(frame, (640, 480))
                obj = DeepFace.analyze(resized_frame, actions=['gender'], enforce_detection=False)
                gender = obj[0]['dominant_gender']
                last_detection_time = current_time
            except Exception as e:
                print("Error de análisis:", e)
                continue

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame_rgb)

        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                
                if gender == 'Man' and lipstick_image is not None:
                    overlay_img = cv2.resize(lipstick_image, (w // 2, h // 5))
                    frame = overlay_image(frame, overlay_img, (x + w // 4, y + int(h * 0.6)))
                
                elif gender == 'Woman' and mustache_image is not None:
                    overlay_img = cv2.resize(mustache_image, (w // 2, h // 6))
                    frame = overlay_image(frame, overlay_img, (x + w // 4, y + int(h * 0.55)))

        cv2.imshow("Filtro de Genero", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

vid.release()
cv2.destroyAllWindows()
